In [2]:
import snapatac2 as snap
import numpy as np
import polars as pl

snap.__version__

'2.4.0'

In [ ]:
import os
os.chdir("../../DATA/Brain_subregion/")

In [4]:
data = snap.pp.import_data(
    fragment_file="GSM6204623_MouseBrain_20um_fragments.tsv.gz",
    genome=snap.genome.mm10,
    file="temp.h5ad",
    sorted_by_barcode=False,
    min_num_fragments=0,
    min_tsse=0
)
data

AnnData object with n_obs x n_vars = 2500 x 0 backed at 'temp.h5ad'
    obs: 'tsse', 'n_fragment', 'frac_dup', 'frac_mito'
    uns: 'reference_sequences'
    obsm: 'insertion'

In [5]:
snap.pp.add_tile_matrix(data)

In [15]:
snap.pp.select_features(data)
snap.tl.spectral(data)

2025-03-16 18:45:37 - INFO - Selected 500000 features.


In [16]:
snap.pp.knn(data)
snap.tl.leiden(data, resolution=0.8)

In [17]:
len(set(data.obs["leiden"]))

7

In [18]:
snap.tl.call_peaks(data, groupby="leiden", q_value=0.01)
peaks = list(data.uns["peaks"]["Peaks"])

2025-03-16 18:45:53 - INFO - Exporting data...
2025-03-16 18:47:17 - INFO - Calling peaks for 7 groups ...
2025-03-16 18:48:02 - INFO - Merging peaks...


In [19]:
import anndata as ad

In [ ]:
peak_mat = snap.pp.make_peak_matrix(data, use_rep=peaks)

In [21]:
np.max(peak_mat.X)

1189

In [ ]:
rna = ad.read_h5ad("rna.h5ad")
peak_mat =peak_mat[rna.obs_names]
peak_mat.obsm["spatial"] = rna.obsm["spatial"].copy()

/tmp/ipykernel_2717297/1009788417.py:6: ImplicitModificationWarning: Setting element `.obsm['spatial']` of view, initializing view as actual.
  peak_mat.obsm["spatial"] = rna.obsm["spatial"]


In [ ]:
peak_mat.write("atac.h5ad")